# Plots of noisy LO transforms
Refer to:


In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import NarrowbandMetadata
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
import torchsig.utils.dsp as dsp
from torchsig.utils.dsp import (
    torchsig_complex_data_type
)

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

In [ ]:
def generate_tone_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

    Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 128.
        scale (int, optional): Scale factor for normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    rng = np.random.default_rng(42)
    sample_rate = 10e6
    
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 10.0,
        snr_db_max = 10.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0, 
        transforms = [],
        target_transforms = [],
        class_list = ['tone'],
        class_distribution = [1.0],
        num_samples = 1,
        seed = 1234
    )
    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 1234
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
# baseband (frequency = 0)
frequency = 0
sample_rate = 4.0
rng = np.random.default_rng(42)

N = 8192
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 

data_white = F.local_oscillator_phase_noise(
    data = tone_bb_data,
    sample_rate = sample_rate,
    frequency = frequency,
    noise_power = 0.01,
    noise_color = 'white',
    rng = rng
)
data_pink = F.local_oscillator_phase_noise(
    data = tone_bb_data,
    sample_rate = sample_rate,
    frequency = frequency,
    noise_power = 0.01,
    noise_color = 'pink',
    rng = rng
)
data_red = F.local_oscillator_phase_noise(
    data = tone_bb_data,
    sample_rate = sample_rate,
    frequency = frequency,
    noise_power = 0.01,
    noise_color = 'red',
    rng = rng
)

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(data_white, Fs=sample_rate, scale='linear', sides='twosided', color='white');
spectrum, freqs, _ = ax.magnitude_spectrum(data_pink, Fs=sample_rate, scale='linear', sides='twosided', color='pink');
spectrum, freqs, _ = ax.magnitude_spectrum(data_red, Fs=sample_rate, scale='linear', sides='twosided', color='red');

ax.set_yscale('log')
# ax.set_xscale('log')
ax.set_xscale('symlog', 
             base=10,          # Logarithm base
             linthresh=1/N)   # Threshold for linear region near zero

# Configure axis labels and grid
ax.set_xlabel('Frequency (Hz) [log]');
ax.set_ylabel('Magnitude [log]');
#plt.ylim([1e-5, None]);

In [ ]:
# frequency = 0.2
rng = np.random.default_rng(42)

frequency = -0.5
sample_rate = 4.0
N = 8192
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 

# mixer = np.exp(2j*np.pi*(frequency/sample_rate)*np.arange(0,len(tone_bb_data)))
# data = tone_bb_data*mixer

data_white = F.local_oscillator_phase_noise(
    data = tone_bb_data,
    sample_rate = sample_rate,
    frequency = frequency,
    noise_power = 0.01,
    noise_color = 'white',
    rng = rng
)
data_pink = F.local_oscillator_phase_noise(
    data = tone_bb_data,
    sample_rate = sample_rate,
    frequency = frequency,
    noise_power = 0.01,
    noise_color = 'pink',
    rng = rng
)
data_red = F.local_oscillator_phase_noise(
    data = tone_bb_data,
    sample_rate = sample_rate,
    frequency = frequency,
    noise_power = 0.01,
    noise_color = 'red',
    rng = rng
)

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()


spectrum, freqs, _ = ax.magnitude_spectrum(data_white, Fs=sample_rate, scale='linear', sides='twosided', color='white');
spectrum, freqs, _ = ax.magnitude_spectrum(data_pink, Fs=sample_rate, scale='linear', sides='twosided', color='pink');
spectrum, freqs, _ = ax.magnitude_spectrum(data_red, Fs=sample_rate, scale='linear', sides='twosided', color='red');

ax.set_yscale('log')
# ax.set_xscale('log')
# ax.set_xscale('symlog', 
#              base=10,          # Logarithm base
#              linthresh=1/N)   # Threshold for linear region near zero

# Configure axis labels and grid
ax.set_xlabel('Frequency (Hz)');
ax.set_ylabel('Magnitude [log]');
#plt.ylim([1e-5, None]);